##### Copyright 2024 Google LLC.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Search tool with Gemini 2.0

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/examples/Search_grounding_for_research_report.ipynb"><img src="https://github.com/google-gemini/cookbook/blob/main/images/colab_logo_32px.png?raw=1" />Run in Google Colab</a>
  </td>
</table>

In this tutorial you are going to leverage the latest [search tool](https://ai.google.dev/gemini-api/docs/models/gemini-v2#search-tool) of the Gemini 2.0 model to write a company report. Note that the search tool is a paid ony feature and this tutorial does not work with a free tier API key.

You may be asking, why does one need to use the search tool for this purpose? Well, as you may be aware, today's business world evolves very fast and LLMs generally are not trained frequently enough to capture the latest updates. Luckily Google search comes to the rescue. Google search is built to provide accurate and nearly realtime information and can help us fulfill this task perfectly.

Note that while Gemini 1.5 models offered the [search grounding](https://ai.google.dev/gemini-api/docs/grounding) feature which may also help you achieve similar results (see [a previous notebook](https://github.com/google-gemini/cookbook/blob/gemini-1.5-archive/examples/Search_grounding_for_research_report.ipynb) using search grounding), the new search tool in Gemini 2.0 models is much more powerful and easier to use, and should be prioritized over search grounding. It is also possible to use multiple tools together (for example, search tool and function calling).

## Setup

### Install SDK

The new **[Google Gen AI SDK](https://ai.google.dev/gemini-api/docs/sdks)** provides programmatic access to Gemini 2.0 (and previous models) using both the [Google AI for Developers](https://ai.google.dev/gemini-api/docs) and [Vertex AI](https://cloud.google.com/vertex-ai/generative-ai/docs/overview) APIs. With a few exceptions, code that runs on one platform will run on both. This means that you can prototype an application using the Developer API and then migrate the application to Vertex AI without rewriting your code.

More details about this new SDK on the [documentation](https://ai.google.dev/gemini-api/docs/sdks) or in the [Getting started](../gemini-2/get_started.ipynb) notebook.

In [9]:
!pip install -U -q google-genai

### Setup your API key

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see [Authentication](../quickstarts/Authentication.ipynb) for an example.

In [10]:
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

### Import the libraries

In [11]:
from google import genai
from google.genai.types import GenerateContentConfig, Tool
from IPython.display import display, HTML, Markdown
import io
import json
import re

### Initialize SDK client

With the new SDK you now only need to initialize a client with you API key (or OAuth if using [Vertex AI](https://cloud.google.com/vertex-ai)). The model is now set in each call.

In [12]:
client = genai.Client(api_key=GOOGLE_API_KEY)

### Select a model

Search tool is a new feature in the Gemini 2.0 model that automatically retrieves accurate and grounded artifacts from the web for developers to further process. Unlike the search grounding in the Gemini 1.5 models, you do not need to set the dynamic retrieval threshold.

For more information about all Gemini models, check the [documentation](https://ai.google.dev/gemini-api/docs/models/gemini) for extended information on each of them.


In [13]:
MODEL_ID = "gemini-2.0-flash" # @param ["gemini-1.5-flash-latest","gemini-2.0-flash-lite-preview-02-05","gemini-2.0-flash","gemini-2.0-pro-preview-02-05"] {"allow-input":true}

## Write the report with Gemini 2.0

### Select a target company to research

Next you will use Alphabet as an example research target.

In [14]:
COMPANY = 'Alphabet' # @param {type:"string"}

### Have Gemini 2.0 plan for the task and write the report

Gemini 2.0 is a huge step up from previous models since it can reason, plan, search and write in one go. In this case, you will only give Gemini 2.0 a prompt to do all of these and the model will finish your task seamlessly. You will also using output streaming, which streams the response as it is being generated, and the model will return chunks of the response as soon as they are generated. If you would like the SDK to return a response after the model completes the entire generation process,  you can use the non-streaming approach as well.

Note that you must enable Google Search Suggestions, which help users find search results corresponding to a grounded response, and [display the search queries](https://ai.google.dev/gemini-api/docs/grounding/search-suggestions#display-requirements) that are included in the grounded response's metadata. You can find [more details](https://ai.google.dev/gemini-api/terms#grounding-with-google-search) about this requirement.

In [16]:
sys_instruction = """You are an analyst that conducts company research.
You are given a company name, and you will work on a company report. You have access
to Google Search to look up company news, updates and metrics to write research reports.

When given a company name, identify key aspects to research, look up that information
and then write a concise company report.

Feel free to plan your work and talk about it, but when you start writing the report,
put a line of dashes (---) to demarcate the report itself, and say nothing else after
the report has finished.
"""

config = GenerateContentConfig(system_instruction=sys_instruction, tools=[Tool(google_search={})], temperature=0)
response_stream = client.models.generate_content_stream(
    model=MODEL_ID, config=config, contents=[COMPANY])

report = io.StringIO()
for chunk in response_stream:
  candidate = chunk.candidates[0]

  for part in candidate.content.parts:
    if part.text:
      display(Markdown(part.text))

      # Find and save the report itself.
      if m := re.search('(^|\n)-+\n(.*)$', part.text, re.M):
          # Find the starting '---' line and start saving.
          report.write(m.group(2))
      elif report.tell():
        # If there's already something in the buffer, keep recording.
        report.write(part.text)

    else:
      print(json.dumps(part.model_dump(exclude_none=True), indent=2))

  # You must enable Google Search Suggestions
  if gm := candidate.grounding_metadata:
    if sep := gm.search_entry_point:
      display(HTML(sep.rendered_content))

Okay

, I will

 research Alphabet and create a company report. Here's my plan:

1

.  **Company Overview:** I'll start with a general overview of Alphabet, including its

 main business segments, mission, and key products/services.
2.  **Recent News and Developments:** I'll look for recent news articles and press

...

 04, 2025.
*   Shares of Alphabet dropped nearly 8% on February 5, 2025, amid worries about whether the tech giant's spending on AI will pay off.
*   Despite recent losses, Alphabet's stock has gained more than 3

0% over the past 12 months.
*   Analysts' opinions on Alphabet Inc (Google) Class C future price: max estimate of 240.00 USD and a min estimate of 184.00 USD.

**Competitive Landscape:**

*   Alphabet operates in a highly competitive

 landscape, facing challenges from companies like Microsoft, Amazon, Apple, Meta, and numerous others across various sectors.
*   In search, Microsoft's Bing directly competes with Google Search. In cloud services, Microsoft's Azure competes with Google Cloud Platform.
*   Alphabet faces intensifying competition in AI technology from both

 established players and emerging startups like OpenAI.

**Future Outlook:**

*   Analysts forecast Alphabet to grow earnings and revenue by 10.8% and 9.7% per annum respectively. EPS is expected to grow by 11.9% per annum.
*   Alphabet's heavy investments in AI and

 other new technologies could result in a temporary slowdown in earnings growth.
*   The company's future profitability outlook is under scrutiny as market analysts have observed that Alphabet's heavy investments in new technologies and business ventures could result in a temporary slowdown in earnings growth.

---

Very impressive! Gemini 2.0 starts by planning for the task, performing relevant searches and streams out a report for you.

### Final output

Render the final output.

In [17]:
display(Markdown(report.getvalue().replace('$', r'\$')))  # Escape $ signs for better MD rendering
display(HTML(sep.rendered_content))

As you can see, the Gemini 2.0 model is able to write a concise, accurate and well-structured research report for us. All the information in the report is factual and up-to-date.

Note that this tutorial is meant to showcase the capability of the new search tool and inspire interesting use cases, not to build a production research report generator. If you are looking to use a tool, please check out Google [Deep Research](https://blog.google/products/gemini/google-gemini-deep-research/) in the Gemini app.

## tech report

In [19]:
TOPIC = 'Mobile App Startup measurement' # @param {type:"string"}

In [20]:
sys_instruction = """You are an analyst researching technology for mobile development solutions. Your task is to provide a concise report that references Apple’s iOS guidelines and Google’s Android guidelines first, along with any other authoritative sources you find relevant.

Feel free to plan and discuss your approach out loud, but when you begin writing your final report, please insert a line of dashes (—) to mark its start.

In your report, do the following:
	1.	Present the most critical best practices or guidelines from both Apple and Google for modern mobile development.
	2.	Propose at least three step-by-step solutions or methods with pros and cons  to implement these guidelines effectively, highlighting potential frameworks or libraries (e.g., Flutter, React Native, SwiftUI, Jetpack Compose).
	3.	Offer additional references or sources for further reading.
"""

config = GenerateContentConfig(system_instruction=sys_instruction, tools=[Tool(google_search={})], temperature=0)
response_stream = client.models.generate_content_stream(
    model=MODEL_ID, config=config, contents=[TOPIC])

report = io.StringIO()
for chunk in response_stream:
  candidate = chunk.candidates[0]

  for part in candidate.content.parts:
    if part.text:
      display(Markdown(part.text))

      # Find and save the report itself.
      if m := re.search('(^|\n)-+\n(.*)$', part.text, re.M):
          # Find the starting '---' line and start saving.
          report.write(m.group(2))
      elif report.tell():
        # If there's already something in the buffer, keep recording.
        report.write(part.text)

    else:
      print(json.dumps(part.model_dump(exclude_none=True), indent=2))

  # You must enable Google Search Suggestions
  if gm := candidate.grounding_metadata:
    if sep := gm.search_entry_point:
      display(HTML(sep.rendered_content))

Okay

, I need

 to research mobile app startup measurement, focusing on guidelines from Apple (iOS) and Google

 (Android). I'll look for best practices and then propose solutions for implementing

 them, considering frameworks like Flutter, React Native, SwiftUI, and Jetpack Compose.

Here's my plan:

1.  **Search for Apple'

...

 to SwiftUI, it uses a declarative approach. Use `remember` and `rememberCoroutineScope` effectively to manage state and asynchronous operations.
*   **Kotlin Coroutines (Android):** For managing background tasks and asynchronous operations in a structured and efficient way.

**3. Additional References and Sources:**

*   **Apple Developer

 Documentation:** [https://developer.apple.com/](https://developer.apple.com/)
*   **Android Developers Documentation:** [https://developer.android.com/](https://developer.android.com/)
*   **React Native Documentation:** [https://reactnative.dev/](https://reactnative

.dev/)
*   **Flutter Documentation:** [https://flutter.dev/](https://flutter.dev/)
*   **WWDC and Google I/O Sessions:** Search for relevant sessions on performance optimization.
*   **Emerge Tools:** [https://emergetools.com/](https://

emergetools.com/) (for performance analysis)
*   **Instabug:** [https://www.instabug.com/](https://www.instabug.com/) (for app performance monitoring)

This report provides a starting point for optimizing mobile app startup time. Continuous monitoring and profiling

 are essential to identify and address performance bottlenecks throughout the app development lifecycle.


# my deep research

In [1]:
from google.colab import drive
from google import genai
from google.genai.types import GenerateContentConfig, Tool
from IPython.display import display, HTML, Markdown
import io
import json
import re
import os
from google.colab import userdata
from datetime import datetime

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

def my_deep_research(topic, sys_instruction):
  """Generates a report using Gemini 2.0, displays it with citations, and saves it to Google Drive.

  Args:
    topic: The topic of the report.
    sys_instruction: The system instruction for Gemini 2.0.
  """

  # Mount Google Drive
  drive.mount('/content/drive')

  # Initialize Gemini client
  client = genai.Client(api_key=GOOGLE_API_KEY)
  MODEL_ID = "gemini-2.0-pro-exp"  # Or your preferred model

  # Configure Gemini with search tool
  config = GenerateContentConfig(
      system_instruction=sys_instruction,
      tools=[Tool(google_search={})],
      temperature=0
  )

  # Generate report content (streaming)
  response_stream = client.models.generate_content_stream(
      model=MODEL_ID, config=config, contents=[topic]
  )

  report = io.StringIO()
  for chunk in response_stream:
    candidate = chunk.candidates[0]

    for part in candidate.content.parts:
      if part.text:
        display(Markdown(part.text))

        # Extract report content
        if m := re.search('(^|\n)-+\n(.*)$', part.text, re.M):
          report.write(m.group(2))
        elif report.tell():
          report.write(part.text)

      else:
        print(json.dumps(part.model_dump(exclude_none=True), indent=2))

    # Display search citations
    if gm := candidate.grounding_metadata:
      if sep := gm.search_entry_point:
        display(HTML(sep.rendered_content))

  # Save the report to Google Drive
  report_content = report.getvalue().replace('$', r'\$')
  timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")  # Format: YYYYMMDD_HHMMSS
  file_path = os.path.join('/content/drive/My Drive/Colab Notebooks/output', f'{topic}_{timestamp}.md')
  with open(file_path, 'w') as f:
    f.write(report_content)

  print(f"Report saved to: {file_path}")


/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: <built-in function any> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


In [2]:
# Example usage
topic = "combine_llm_with_notion"

sys_instruction = """
You are an efficiency fan and expert in LLM python, chacing best chance to improve
personal efficicienty by using the latest llm technology.

1. search online to find out the best practice.
2. check out the most reliable code source and present demo python snippets.
3. address the question: how to find out page_id in notion.
4. Feel free to plan your work and talk about it, but when you start writing the report,
put a line of dashes (---) to demarcate the report itself, and say nothing else after
the report has finished.
5. add citations of web link most relevant to the question.
6. output in markdown format.
""" # [Your detailed instructions here]


my_deep_research(topic, sys_instruction)

Mounted at /content/drive


Okay, I

 will help you to find out the best practice to combine LLM with Notion. I

 will follow these steps:

1.  **Search online:** Use Google Search to find the

 best practices, reliable code sources, and tutorials on integrating LLM with Notion.
2.  **Code examples:** Identify and present demo Python snippets for interacting

...


**4. Creating a page in database (using `notion_client`):**

```python
from notion_client import Client

notion = Client(auth="secret_YOUR_INTEGRATION_TOKEN")
database_id = "YOUR_DATABASE_ID" # get database id from url

def create_page_in_database(database_id:str, title:str, content:str):
    new_page_data = {
        "parent": {"database_id": database_id},
        "properties": {
            "Name": {  # Assuming your database has a "Name" property (Title type)
                "title": [{"text": {"content": title}}]
            },
        },
        "children":[
            {
                "object": "block",
                "type": "paragraph",
                "paragraph": {
                    "rich_text": [
                        {
                            "type": "text",
                            "text": {
                                "content": content
                            }
                        }
                    ]
                }
            }
        ]
    }
    created_page = notion.pages.create(**new_page_data)
    return created_page

# create a new page
create_page_in_database(database_id, "My New Page Title", "This is the content of my new page.")
```



These snippets provide a starting point for integrating LLMs with Notion.  You can expand upon these examples to build more complex applications, such as creating entire workflows triggered by natural language commands processed by an LLM. Remember to consult

 the official Notion API documentation for the most up-to-date information and capabilities.

Most relevant web link: [2] Working with page content - Notion API


Report saved to: /content/drive/My Drive/combine_llm_with_notion_20250207_232056.md


## Next Steps

* To learn more about the search tool, check out the [Search tool documentation](https://ai.google.dev/gemini-api/docs/models/gemini-v2#search-tool).
* To get started with streaming, check out the [Streaming Quickstart](../quickstarts/Streaming.ipynb).
* To get started with the search tool, check out the [Search tool quick start](../quickstarts/Search_Grounding.ipynb).
* To see how much easier and more powerful the search tool in Gemini 2.0 is than the previous search grounding feature, compare this tutorial with the [previous example](../examples/Search_grounding_for_research_report.ipynb).
* To learn more about all the new Gemini 2.0 features, check out the [Gemini 2.0 quickstart notebooks](https://github.com/google-gemini/cookbook/tree/main/gemini-2/).